In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils_1_1

import pandas as pd
import numpy as np
import altair as alt
from altair_saver import save
import datetime
import dateutil.parser

from constants_1_1 import SITE_FILE_TYPES
from utils_1_1 import (
    read_loinc_df,
    get_site_file_paths,
    get_site_file_info,
    get_site_ids,
    read_full_daily_counts_df,
    get_visualization_subtitle,
    get_country_color_map,
    apply_theme,
    merge_single_site_country_adult_name,
)
from web import for_website

alt.data_transformers.disable_max_rows(); # Allow using rows more than 5000

In [ ]:
DATA_RELEASE = "2020-09-28"
COHORT = "Adult"
MERGE_SINGLE_SITE_COUNTRIES = True

In [ ]:
df = read_full_daily_counts_df()
df.head()

## Remove pediatric sites

In [ ]:
df = df.loc[df["pediatric"] == False]
df = df.drop(columns=["pediatric"])
df.head()

In [ ]:
# Get number of sites after restricting to pediatrics
NUM_SITES = len(df["siteid"].unique().tolist())

In [ ]:
df = df.replace(-99, np.nan)
df = df.replace(-999, np.nan)

In [ ]:
df["num_sites"] = 1

## If site is missing data for a particular date, use the most recent previous data point for that date

In [ ]:
max_date = df["calendar_date"].max()
max_date_str = str(max_date).split(" ")[0]

all_date_country_df = pd.DataFrame()
for siteid, cd_df in df.groupby(["siteid"]):
    min_date = cd_df["calendar_date"].min()
    min_date_str = str(min_date).split(" ")[0]
    
    num_days = (dateutil.parser.parse(max_date_str) - dateutil.parser.parse(min_date_str)).days
    
    cd_df = cd_df.copy()
    cd_df["calendar_date"] = cd_df["calendar_date"].astype(str)
    
    prev_date_row = None
    for day_offset in range(num_days):
        curr_date = dateutil.parser.parse(min_date_str) + datetime.timedelta(days=day_offset)
        curr_date_str = str(curr_date).split(" ")[0]
        
        try:
            curr_date_row = cd_df.loc[cd_df["calendar_date"] == curr_date_str].to_dict('records')[0]
            prev_date_row = curr_date_row
        except:
            prev_date_row['calendar_date'] = curr_date_str
            prev_date_row['num_sites'] = 0
            cd_df = cd_df.append(prev_date_row, ignore_index=True)
    
    all_date_country_df = all_date_country_df.append(cd_df, ignore_index=True)
def convert_date(date_str):
    try:
        return dateutil.parser.parse(date_str)
    except:
        return np.nan
all_date_country_df["calendar_date"] = all_date_country_df["calendar_date"].apply(convert_date)
df = all_date_country_df

In [ ]:
df["num_patients_in_hospital_minus_severe_on_this_date"] = df["num_patients_in_hospital_on_this_date"] - df["num_patients_in_hospital_and_severe_on_this_date"]

In [ ]:
country_color_map = get_country_color_map(merge_single_site_countries=MERGE_SINGLE_SITE_COUNTRIES)

In [ ]:
plot = alt.Chart(df).mark_line().encode(
    x=alt.X("calendar_date", axis=alt.Axis(title="Date")),
    y=alt.Y("num_patients_in_hospital_on_this_date:Q", axis=alt.Axis(title="Number of Patients in Hospital")),
    color=alt.Color("siteid:N", legend=alt.Legend(title="Site"))
).properties(title={
    "text": ["Number of Hospitalized Patients by Site"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE, num_sites=NUM_SITES, cohort=COHORT),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot)

plot

In [ ]:
if MERGE_SINGLE_SITE_COUNTRIES:
    df["country"] = df["country"].apply(merge_single_site_country_adult_name)

country_sum_df = df.groupby(["country", "calendar_date"]).sum().reset_index()
country_sum_df.head()

## Convert cumulative counts to daily counts

Since the same patient may be in the hospital on multiple dates, we want to use the cumulative counts to find the daily hospitalization rate.

In [ ]:

country_sum_temp_df = pd.DataFrame(index=[], data=[], columns=[])
for country, country_df in country_sum_df.groupby("country"):
    country_df = country_df.copy()
    country_df["cum_diff_all"] = np.concatenate((np.array([np.nan]), np.diff(country_df["cumulative_patients_all"].values)))
    country_df["cum_diff_severe"] = np.concatenate((np.array([np.nan]), np.diff(country_df["cumulative_patients_severe"].values)))
    country_df["cum_diff_dead"] = np.concatenate((np.array([np.nan]), np.diff(country_df["cumulative_patients_dead"].values)))
    country_df["cum_diff_all_minus_severe"] = country_df["cum_diff_all"] - country_df["cum_diff_severe"]
    
    country_df["cum_diff_all"] = country_df["cum_diff_all"].clip(lower=0)
    country_df["cum_diff_severe"] = country_df["cum_diff_severe"].clip(lower=0)
    country_df["cum_diff_dead"] = country_df["cum_diff_dead"].clip(lower=0)
    country_df["cum_diff_all_minus_severe"] = country_df["cum_diff_all_minus_severe"].clip(lower=0)
    """
    country_df["count"] = np.concatenate((np.array([np.nan]), np.diff(country_df["cumulative_count"].values)))
    country_df["cumulative_count"] = country_df["cumulative_count"].replace(0, np.nan)
    
    country_df["N0"] = country_df["cumulative_count"].shift(1) # N0 is the total case up to the day before
    country_df["n1"] = country_df["count"] # n1 is the case number this day
    country_df["n2"] = country_df["n1"].shift(1) # n2 is the case number yesterday
    
    country_df["percent_increase"] = (country_df["n1"] / country_df["N0"]) * 100
    
    country_df['R'] = country_df["percent_increase"] # TODO: is this correct?
    # TODO: update CI formula
    country_df['C'] = country_df['R'] - 1
    country_df['standard_error'] = country_df.apply(lambda obs: (obs['R']+np.power(obs['R'], 2))/obs['n2'], axis='columns')
    country_df['95_CI_below'] = country_df.apply(lambda obs: obs['C'] - 1.96*np.sqrt(obs['standard_error']), axis='columns')
    country_df['95_CI_above'] = country_df.apply(lambda obs: obs['C'] + 1.96*np.sqrt(obs['standard_error']), axis='columns')
    country_df = country_df.replace([np.inf, -np.inf], np.nan)
    """

    country_sum_temp_df = country_sum_temp_df.append(country_df, ignore_index=True)
country_sum_df = country_sum_temp_df
country_sum_df.tail()


In [ ]:
country_sum_molten_df = country_sum_df.melt(id_vars=["country", "calendar_date", "num_sites"])
country_sum_molten_df.head()

# Daily counts by country, with country as color, variable as dropdown

In [ ]:
CATEGORIES = country_sum_molten_df["variable"].unique().tolist()

country_color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values()))

VAR_NAME_MAP = {
    "num_patients_in_hospital_on_this_date": "Number of Patients in Hospital",
    "num_patients_in_hospital_minus_severe_on_this_date": "Number of Patients minus Number of Severe Patients in Hospital",
    "num_patients_in_hospital_and_severe_on_this_date": "Number of Severe Patients in Hospital",
    "cumulative_patients_all": "Cumulative Number of Patients",
    "cumulative_patients_severe": "Cumulative Number of Patients Severe",
    "cumulative_patients_dead": "Cumulative Number of Patients Dead",
    "cum_diff_all": "Number of Hospitalized Patients",
    "cum_diff_severe": "Number of Hospitalized Patients Severe",
    "cum_diff_dead": "Number of Hospitalized Patients Dead",
    "cum_diff_all_minus_severe": "Number of Hospitalized Patients All minus Severe",
}

var_country_sum_molten_df = country_sum_molten_df.copy()
var_country_sum_molten_df["variable"] = var_country_sum_molten_df["variable"].apply(lambda x: VAR_NAME_MAP[x])

CATEGORIES = var_country_sum_molten_df["variable"].unique().tolist()

dailycount_dropdown = alt.binding_select(options=CATEGORIES)
dailycount_selection = alt.selection_single(fields=["variable"], bind=dailycount_dropdown, name="Value", init={"variable": VAR_NAME_MAP["num_patients_in_hospital_on_this_date"]})

filtered_plot = alt.Chart(var_country_sum_molten_df).transform_filter(
    dailycount_selection
)

tooltip = [
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("calendar_date", title="Date"),
    alt.Tooltip("variable", title="Variable"),
    alt.Tooltip("value", title="Value"),
    alt.Tooltip("num_sites", title="Number of sites"),
]

top_plot = filtered_plot.mark_line().encode(
    x=alt.X("calendar_date", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Number of patients")),
    color=alt.Color("country:N", legend=alt.Legend(title="Country", labelLimit=300), scale=country_color_scale),
    tooltip=tooltip
)

bottom_plot = filtered_plot.mark_bar(size=1.5).encode(
    x=alt.X("calendar_date", axis=alt.Axis(title="Date")),
    y=alt.Y("num_sites:Q", axis=alt.Axis(title="# of sites")),
    color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale),
    tooltip=tooltip
).properties(height=80)

plot = alt.vconcat(top_plot, bottom_plot).resolve_scale(x="shared", color="shared").properties(title={
    "text": ["Daily Counts by Country"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot).add_selection(
    dailycount_selection
)

for_website(plot, f"Daily Counts {COHORT}", "Daily counts by country with variable dropdown", df=var_country_sum_molten_df)

plot


# Daily counts by country, with `num_patients_in_hospital_minus_severe_on_this_date` and `num_patients_in_hospital_and_severe_on_this_date` as color, country as dropdown

In [ ]:
COUNTRIES = country_sum_molten_df["country"].unique().tolist()

num_in_hospital_by_country_molten_df = country_sum_molten_df.loc[country_sum_molten_df["variable"].isin([
    "num_patients_in_hospital_minus_severe_on_this_date",
    "num_patients_in_hospital_and_severe_on_this_date"
])].copy()
num_in_hospital_by_country_molten_df["variable"] = num_in_hospital_by_country_molten_df["variable"].replace({
    "num_patients_in_hospital_minus_severe_on_this_date": "All minus Severe",
    "num_patients_in_hospital_and_severe_on_this_date": "Severe",
})

country_color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values()))
severity_color_scale = alt.Scale(domain=["All minus Severe", "Severe"], range=["#A9A9A9", "#000000"])

country_dropdown = alt.binding_select(options=COUNTRIES)
country_selection = alt.selection_single(fields=["country"], bind=country_dropdown, name="Country", init={"country": "USA"})

filtered_plot = alt.Chart(num_in_hospital_by_country_molten_df).transform_filter(
    country_selection
)

tooltip = [
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("calendar_date", title="Date"),
    alt.Tooltip("variable", title="Variable"),
    alt.Tooltip("value", title="Number of patients"),
    alt.Tooltip("num_sites", title="Number of sites"),
]

top_plot = filtered_plot.mark_line().encode(
    x=alt.X("calendar_date", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Number of patients in hospital")),
    color=alt.Color("variable:N", legend=alt.Legend(title="Severity"), scale=severity_color_scale),
    tooltip=tooltip
)

bottom_plot = filtered_plot.mark_bar(size=1.5).encode(
    x=alt.X("calendar_date", axis=alt.Axis(title="Date")),
    y=alt.Y("num_sites:Q", axis=alt.Axis(title="# of sites")),
    color=alt.Color("country:N", legend=None, scale=country_color_scale),
    tooltip=tooltip
).properties(height=80)

plot = alt.vconcat(top_plot, bottom_plot).resolve_scale(x="shared", color="independent").properties(title={
    "text": ["Daily Counts by Country and Severity"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot).add_selection(
    country_selection
)

for_website(plot, f"Daily Counts {COHORT}", "Daily counts by severity with country dropdown", df=num_in_hospital_by_country_molten_df)

plot

# Daily counts by country, with `cumulative_patients_all`, `cumulative_patients_severe`, `cumulative_patients_dead` as color, country as dropdown

In [ ]:
COUNTRIES = country_sum_molten_df["country"].unique().tolist()

cumulative_by_country_molten_df = country_sum_molten_df.loc[country_sum_molten_df["variable"].isin([
    "cumulative_patients_all",
    "cumulative_patients_severe",
    "cumulative_patients_dead"
])].copy()
cumulative_by_country_molten_df["variable"] = cumulative_by_country_molten_df["variable"].replace({
    "cumulative_patients_all": "All",
    "cumulative_patients_severe": "Severe",
    "cumulative_patients_dead": "Dead",
})

country_color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values()))
severity_color_scale = alt.Scale(domain=["All minus Severe", "Severe"], range=["#A9A9A9", "#000000"])

country_dropdown = alt.binding_select(options=COUNTRIES)
country_selection = alt.selection_single(fields=["country"], bind=country_dropdown, name="Country", init={"country": "USA"})

filtered_plot = alt.Chart(cumulative_by_country_molten_df).transform_filter(
    country_selection
)

tooltip = [
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("calendar_date", title="Date"),
    alt.Tooltip("variable", title="Variable"),
    alt.Tooltip("value", title="Cumulative number of patients"),
    alt.Tooltip("num_sites", title="Number of sites"),
]

top_plot = filtered_plot.mark_line().encode(
    x=alt.X("calendar_date", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Cumulative number of patients")),
    color=alt.Color("variable:N", legend=alt.Legend(title="Severity")),
    tooltip=tooltip
)

bottom_plot = filtered_plot.mark_bar(size=1.5).encode(
    x=alt.X("calendar_date", axis=alt.Axis(title="Date")),
    y=alt.Y("num_sites:Q", axis=alt.Axis(title="# of sites")),
    color=alt.Color("country:N", legend=None, scale=country_color_scale),
    tooltip=tooltip
).properties(height=80)

plot = alt.vconcat(top_plot, bottom_plot).resolve_scale(x="shared", color="independent").properties(title={
    "text": ["Cumulative Daily Counts by Country and Severity"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot).add_selection(
    country_selection
)

for_website(plot, f"Daily Counts {COHORT}", "Cumulative daily counts by severity with country dropdown", df=cumulative_by_country_molten_df)

plot

# Daily counts by country, with `num_patients_in_hospital_minus_severe_on_this_date` and `num_patients_in_hospital_and_severe_on_this_date` as color, country as column facet

In [ ]:
COUNTRIES = country_sum_molten_df["country"].unique().tolist()

column_width = 200

num_in_hospital_by_country_molten_df = country_sum_molten_df.loc[country_sum_molten_df["variable"].isin([
    "num_patients_in_hospital_minus_severe_on_this_date",
    "num_patients_in_hospital_and_severe_on_this_date"
])].copy()
num_in_hospital_by_country_molten_df["variable"] = num_in_hospital_by_country_molten_df["variable"].replace({
    "num_patients_in_hospital_minus_severe_on_this_date": "All minus Severe",
    "num_patients_in_hospital_and_severe_on_this_date": "Severe",
})

country_color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values()))
severity_color_scale = alt.Scale(domain=["All minus Severe", "Severe"], range=["#A9A9A9", "#000000"])


filtered_plot = alt.Chart(num_in_hospital_by_country_molten_df)

tooltip = [
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("calendar_date", title="Date"),
    alt.Tooltip("variable", title="Variable"),
    alt.Tooltip("value", title="Number of patients"),
    alt.Tooltip("num_sites", title="Number of sites"),
]

top_plot = filtered_plot.mark_line().encode(
    x=alt.X("calendar_date", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Number of patients in hospital")),
    color=alt.Color("variable:N", legend=alt.Legend(title="Severity"), scale=severity_color_scale),
    tooltip=tooltip
).properties(width=column_width).facet(
    column=alt.Column(
        "country:N",
        sort=COUNTRIES,
        header=alt.Header(title=None)
    )
)

bottom_plot = filtered_plot.mark_bar(size=1.5).encode(
    x=alt.X("calendar_date", axis=alt.Axis(title="Date")),
    y=alt.Y("num_sites:Q", axis=alt.Axis(title="# of sites")),
    #color=alt.Color("country:N", legend=alt.Legend(title="Country"), scale=country_color_scale),
    tooltip=tooltip
).properties(height=80, width=column_width).facet(
    column=alt.Column(
        "country:N",
        sort=COUNTRIES,
        header=alt.Header(title=None, labels=False)
    )
)

plot = alt.vconcat(top_plot, bottom_plot).resolve_scale(x="shared", color="independent").properties(title={
    "text": ["Daily Counts by Country and Severity"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot)

for_website(plot, f"Daily Counts {COHORT}", "Daily counts by country and severity", df=num_in_hospital_by_country_molten_df)

plot

# Daily counts by country, with `cumulative_patients_all`, `cumulative_patients_severe`, `cumulative_patients_dead` as color, country as column facet

In [ ]:
COUNTRIES = country_sum_molten_df["country"].unique().tolist()

column_width = 200

cumulative_by_country_molten_df = country_sum_molten_df.loc[country_sum_molten_df["variable"].isin([
    "cumulative_patients_all",
    "cumulative_patients_severe",
    "cumulative_patients_dead"
])].copy()
cumulative_by_country_molten_df["variable"] = cumulative_by_country_molten_df["variable"].replace({
    "cumulative_patients_all": "All",
    "cumulative_patients_severe": "Severe",
    "cumulative_patients_dead": "Dead",
})

country_color_scale = alt.Scale(domain=list(country_color_map.keys()), range=list(country_color_map.values()))
severity_color_scale = alt.Scale(domain=["All minus Severe", "Severe"], range=["#A9A9A9", "#000000"])

filtered_plot = alt.Chart(cumulative_by_country_molten_df)

tooltip = [
    alt.Tooltip("country", title="Country"),
    alt.Tooltip("calendar_date", title="Date"),
    alt.Tooltip("variable", title="Variable"),
    alt.Tooltip("value", title="Cumulative number of patients"),
    alt.Tooltip("num_sites", title="Number of sites"),
]

top_plot = filtered_plot.mark_line().encode(
    x=alt.X("calendar_date", axis=alt.Axis(title=None)),
    y=alt.Y("value:Q", axis=alt.Axis(title="Cumulative number of patients")),
    color=alt.Color("variable:N", legend=alt.Legend(title="Severity")),
    tooltip=tooltip
).properties(width=column_width).facet(
    column=alt.Column(
        "country:N",
        sort=COUNTRIES,
        header=alt.Header(title=None)
    )
)

bottom_plot = filtered_plot.mark_bar(size=1.5).encode(
    x=alt.X("calendar_date", axis=alt.Axis(title="Date")),
    y=alt.Y("num_sites:Q", axis=alt.Axis(title="# of sites")),
    #color=alt.Color("country:N", legend=None, scale=country_color_scale),
    tooltip=tooltip
).properties(height=80, width=column_width).facet(
    column=alt.Column(
        "country:N",
        sort=COUNTRIES,
        header=alt.Header(title=None, labels=False)
    )
)

plot = alt.vconcat(top_plot, bottom_plot).resolve_scale(x="shared", color="independent").properties(title={
    "text": ["Cumulative Daily Counts by Country and Severity"], 
    "dx": 50,
    "subtitle": get_visualization_subtitle(data_release=DATA_RELEASE),
    "subtitleColor": "gray",
    "anchor": "middle",
})

plot = apply_theme(plot)

for_website(plot, f"Daily Counts {COHORT}", "Cumulative daily counts by country and severity", df=cumulative_by_country_molten_df)

plot